In [34]:
import os
import shutil 
import cv2
import glob
import numpy as np
import random 
from sklearn.model_selection import train_test_split

random.seed(10)

In [35]:
def augment_image(image, hue_shift, saturation_scale, brightness_scale):
    # Convert the image from BGR to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply augmentation
    hsv_image[:, :, 0] = (hsv_image[:, :, 0] + hue_shift) % 180
    hsv_image[:, :, 1] = np.clip(saturation_scale * hsv_image[:, :, 1], 0, 255)
    hsv_image[:, :, 2] = np.clip(brightness_scale * hsv_image[:, :, 2], 0, 255)

    # Convert the augmented image back to BGR
    augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

    return augmented_image

# Set augmentation parameters
hue_shifts = [20, -20, 0, 10, 15]  # Degrees to shift the hue
saturation_scales = [1.5, 0.5, 1.0, 1.2, 1.4]  # Scaling factor for saturation
brightness_scales = [1.2, 1.0, 0.8, 1.0, 1.2]  # Scaling factor for brightness

# Adjusting the function to ensure the default sizes sum up to 1.0
def split_img_paths(img_paths, train_size=0.7, val_size=0.2, test_size=0.1):
    """
    Split a list of image paths into training, validation, and test sets.

    Parameters:
    img_paths (list): List of image paths.
    train_size (float): Proportion of the dataset to include in the train split.
    val_size (float): Proportion of the dataset to include in the validation split.
    test_size (float): Proportion of the dataset to include in the test split.

    Returns:
    train_paths, val_paths, test_paths: Lists of image paths for training, validation, and test sets.
    """

    # if train_size + val_size + test_size != 1.0:
    #     raise ValueError("Train, validation, and test sizes must sum to 1.")

    # Splitting the data into training and temp (val + test) sets
    train_paths, temp_paths = train_test_split(img_paths, train_size=train_size)

    # Splitting the temp set into validation and test sets
    val_size_adjusted = val_size / (val_size + test_size)  # Adjusting validation size for the temp set
    val_paths, test_paths = train_test_split(temp_paths, train_size=val_size_adjusted)

    return train_paths, val_paths, test_paths

In [32]:
classes = ['lateralRaises', 'squats', 'pushups']
img_dir = "C://Users//mirza//ASU//MS_master_folder//semester_2//cenembedded//finalProject//customDataset_org//"
output_dir = 'data/'
for each in classes:
    for every in ["train", 'val', 'test']:
        os.makedirs( output_dir + '/' + every + '/' + each, )
modes = ["train", 'val', 'test']

In [36]:
for each_cls in classes:
    img_files = glob.glob( img_dir + '/' + each_cls + '/*/*.jpg')
    # print( len( img_files))
    random.shuffle(img_files)
    train_img_files, val_img_files, test_img_files = split_img_paths( img_files)
    count = 0
    for eachMode in [train_img_files, val_img_files, test_img_files]:
        for idx, each_path in enumerate( eachMode):        
            img = cv2.imread( each_path)        
            for i, (hue_shift, saturation_scale, brightness_scale) in enumerate(zip(hue_shifts, saturation_scales, brightness_scales)):
                augmented_image = augment_image(img, hue_shift, saturation_scale, brightness_scale)
                cv2.imwrite( output_dir + modes[count] + '/' + each_cls + '/' + f'augmented_image_{i+1}_{each_cls}_{idx}.jpg', augmented_image)        
        count += 1